# Preprocessing
- Stem the Data (run/running/ran --> ran)
    - Data is stemmed
    - lower case
    - lemotizor - reduce inflection to base form rather than chopping off
- Identify the most commonly used words in decription
- One hot encody top top 10(ish) of the nontrvial words
- Scale the Data
- PCA (95% of variation captured)

# Models

- [Linear Regression](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LinearRegression.html)

    - There are not many parameters to set

    - I dont think feature extraction will work well on our catagorical dataset
- [Decision Tree Regression](https://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeRegressor.html)

    - Parameters
        - Criterion {“mse”, “friedman_mse”, “mae”}
        - Max Depth (int)
        - Min Leaf Samples (int)
        - max_features int, float or {“auto”, “sqrt”, “log2”}
            - This one might not be as useful
    - [Plot Tree Regression](https://scikit-learn.org/stable/auto_examples/tree/plot_tree_regression.html)
- [Perceptron] https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.Perceptron.html
     - Parameters
        - Penalty  {‘l2’,’l1’,’elasticnet’}, default=None
        - verbose- the verbosity level
            - not sure what this is
        -validation_fraction
        -n_iter_no_chang


- Neural Network

# Set Hyperparameters

- Use 10-fold cross validation to tune hyperparameters
    - using RSS as an error metric?
- [Grid Search](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html)

In [73]:
import pandas as pd
import numpy as np
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
import string  
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split
from tqdm import tqdm #pip install tqdm

In [74]:

df = pd.read_csv("data/winemag-data-130k-v2.csv", index_col=0)

In [75]:
df

,country,description,designation,points,price,province,region_1,region_2,taster_name,taster_twitter_handle,title,variety,winery
0,Italy,"Aromas include tropical fruit, broom, brimston...",Vulkà Bianco,87,NaN,Sicily & Sardinia,Etna,NaN,Kerin O’Keefe,@kerinokeefe,Nicosia 2013 Vulkà Bianco (Etna),White Blend,Nicosia
1,Portugal,"This is ripe and fruity, a wine that is smooth...",Avidagos,87,15.0,Douro,NaN,NaN,Roger Voss,@vossroger,Quinta dos Avidagos 2011 Avidagos Red (Douro),Portuguese Red,Quinta dos Avidagos
2,US,"Tart and snappy, the flavors of lime flesh and...",NaN,87,14.0,Oregon,Willamette Valley,Willamette Valley,Paul Gregutt,@paulgwine,Rainstorm 2013 Pinot Gris (Willamette Valley),Pinot Gris,Rainstorm
3,US,"Pineapple rind, lemon pith and orange blossom ...",Reserve Late Harvest,87,13.0,Michigan,Lake Michigan Shore,NaN,Alexander Peartree,NaN,St. Julian 2013 Reserve Late Harvest Riesling ...,Riesling,St. Julian
4,US,"Much like the regular bottling from 2012, this...",Vintner's Reserve Wild Child Block,87,65.0,Oregon,Willamette Valley,Willamette Valley,Paul Gregutt,@paulgwine,Sweet Cheeks 2012 Vintner's Reserve Wild Child...,Pinot Noir,Sweet Cheeks
...,...,...,...,...,...,...,...,...,...,...,...,...,...
129966,Germany,Notes of honeysuckle and cantaloupe sweeten th...,Brauneberger Juffer-Sonnenuhr Spätlese,90,28.0,Mosel,NaN,NaN,Anna Lee C. Iijima,NaN,Dr. H. Thanisch (Erben Müller-Burggraef) 2013 ...,Riesling,Dr. H. Thanisch (Erben Müller-Burggraef)
129967,US,Citation is given as much as a decade of bottl...,NaN,90,75.0,Oregon,Oregon,Oregon Other,Paul Gregutt,@paulgwine,Citation 2004 Pinot Noir (Oregon),Pinot Noir,Citation
129968,France,Well-drained gravel soil gives this wine its c...,Kritt,90,30.0,Alsace,Alsace,NaN,Roger Voss,@vossroger,Domaine Gresser 2013 Kritt Gewurztraminer (Als...,Gewürztraminer,Domaine Gresser
129969,France,"A dry style of Pinot Gris, this is crisp with ...",NaN,90,32.0,Alsace,Alsace,NaN,Roger Voss,@vossroger,Domaine Marcel Deiss 2012 Pinot Gris (Alsace),Pinot Gris,Domaine Marcel Deiss


In [76]:
for attr in df:
    print(f"{attr}: {round((sum(df[attr].isnull())/df.shape[0])*100,4)}%")

country: 0.0485%
description: 0.0%
designation: 28.8257%
points: 0.0%
price: 6.9215%
province: 0.0485%
region_1: 16.3475%
region_2: 61.1367%
taster_name: 20.1922%
taster_twitter_handle: 24.0154%
title: 0.0%
variety: 0.0008%
winery: 0.0%


In [77]:

def text_stemming(df):
    stop_words = set(stopwords.words("english"))
    lemmatizer = WordNetLemmatizer()
    xFeat = df.to_numpy()
    newDescription = []
    l = len(xFeat)
    for i in tqdm(range (l), desc="Stemming"):
        wordList = word_tokenize((xFeat[i].lower()).translate(str.maketrans('', '', string.punctuation))) #splits the words into a list, and lowercases them
        word = ""
        for item in wordList:
            if item not in stop_words:#we can get rid of stop words: "is", "a", "the"
                word += lemmatizer.lemmatize(item) #replace each word with its stem
                word += " "
        newDescription.append(word)
    return pd.DataFrame(newDescription)

In [78]:
df["description"] = text_stemming(df["description"])

Stemming: 100%|██████████| 129971/129971 [02:03<00:00, 1049.01it/s]


In [79]:
df.dropna(subset=['price'], inplace=True)

In [80]:
df

,country,description,designation,points,price,province,region_1,region_2,taster_name,taster_twitter_handle,title,variety,winery
1,Portugal,"ripe fruity , wine smooth still structured . f...",Avidagos,87,15.0,Douro,NaN,NaN,Roger Voss,@vossroger,Quinta dos Avidagos 2011 Avidagos Red (Douro),Portuguese Red,Quinta dos Avidagos
2,US,"tart snappy , flavor lime flesh rind dominate ...",NaN,87,14.0,Oregon,Willamette Valley,Willamette Valley,Paul Gregutt,@paulgwine,Rainstorm 2013 Pinot Gris (Willamette Valley),Pinot Gris,Rainstorm
3,US,"pineapple rind , lemon pith orange blossom sta...",Reserve Late Harvest,87,13.0,Michigan,Lake Michigan Shore,NaN,Alexander Peartree,NaN,St. Julian 2013 Reserve Late Harvest Riesling ...,Riesling,St. Julian
4,US,"much like regular bottling 2012 , come across ...",Vintner's Reserve Wild Child Block,87,65.0,Oregon,Willamette Valley,Willamette Valley,Paul Gregutt,@paulgwine,Sweet Cheeks 2012 Vintner's Reserve Wild Child...,Pinot Noir,Sweet Cheeks
5,Spain,blackberry raspberry aroma show typical navarr...,Ars In Vitro,87,15.0,Northern Spain,Navarra,NaN,Michael Schachner,@wineschach,Tandem 2011 Ars In Vitro Tempranillo-Merlot (N...,Tempranillo-Merlot,Tandem
...,...,...,...,...,...,...,...,...,...,...,...,...,...
129966,Germany,note honeysuckle cantaloupe sweeten deliciousl...,Brauneberger Juffer-Sonnenuhr Spätlese,90,28.0,Mosel,NaN,NaN,Anna Lee C. Iijima,NaN,Dr. H. Thanisch (Erben Müller-Burggraef) 2013 ...,Riesling,Dr. H. Thanisch (Erben Müller-Burggraef)
129967,US,citation given much decade bottle age prior re...,NaN,90,75.0,Oregon,Oregon,Oregon Other,Paul Gregutt,@paulgwine,Citation 2004 Pinot Noir (Oregon),Pinot Noir,Citation
129968,France,well-drained gravel soil give wine crisp dry c...,Kritt,90,30.0,Alsace,Alsace,NaN,Roger Voss,@vossroger,Domaine Gresser 2013 Kritt Gewurztraminer (Als...,Gewürztraminer,Domaine Gresser
129969,France,"dry style pinot gris , crisp acidity . also we...",NaN,90,32.0,Alsace,Alsace,NaN,Roger Voss,@vossroger,Domaine Marcel Deiss 2012 Pinot Gris (Alsace),Pinot Gris,Domaine Marcel Deiss


In [81]:
 xTrain, xTest, yTrain, yTest = train_test_split(df.loc[:, df.columns != 'price'], df['price'], test_size =.3, random_state = 334)